In [1]:
# !pip install wandb
# !pip install transformers
# !pip install sentencepiece
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "0,2,3" 
import pandas as pd
import numpy as np
import re
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import math
import torch.optim as optim
import wandb


wandb.login()
%env WANDB_PROJECT=NEL
# wandb.init(project="NEL")

# os.environ["WANDB_DISABLED"] = "true"


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# device = torch.device("cuda:2") if torch.cuda.is_available() else torch.device("cpu")


wandb: Currently logged in as: hodz199. Use `wandb login --relogin` to force relogin


env: WANDB_PROJECT=NEL


In [2]:
# Create torch dataset
# https://towardsdatascience.com/fine-tuning-pretrained-nlp-models-with-huggingfaces-trainer-6326a4456e7b

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels['input_ids'][idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [3]:
max_source_length = 1024
max_target_length = 128

In [4]:
from transformers import T5TokenizerFast, T5ForConditionalGeneration 
from transformers import EarlyStoppingCallback

tokenizer = T5TokenizerFast.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base").to(device)

In [5]:
training_data = pd.read_csv('./2-NEL_Data/2-csv_format_2/training_data_shuffled.csv')
training_data = training_data.sample(frac=1, random_state=1)
training_data

,qid,question,entity,wikidata_reply,qid_in_reply,input_len
4312,Q7492362,** Who was involved in the sheemore ambush? **,** sheemore ambush **,"** [[ Q7492362 , Sheemore ambush , ambush duri...",True,155
12486,Q82955,** name a professional politician. **,** politician **,"** [[ Q51556674 , Politician , song by Cream ]...",True,567
92904,Q738653,** white can play either a3 or a4 (see algebr...,** algebraic notation **,"** [[ Q60418499 , Algebraic Notation of Kinshi...",True,650
20220,Q4776569,** Where in europe was antonio demo born **,** antonio demo **,"** [[ Q4776569 , Antonio Demo , Italian-Americ...",True,126
105441,Q1297,"** Filming. Prior to filming, Mendes sought t...",** Chicago **,"** [[ Q2233885 , Willard , city in Huron Count...",True,665
...,...,...,...,...,...,...
98047,Q816704,** the Daily Mirror's new magazine. Their Dep...,** pâté **,"** [[ Q1044124 , paten , small plate used to h...",True,826
5192,Q2247706,** who directed the crowd roars **,** the crowd roars **,"** [[ Q2247706 , The Crowd Roars , 1938 film b...",True,244
77708,Q100047,** AmIRC is an MUI-based IRC client for the Am...,** Amiga **,"** [[ Q20049564 , Amiga , song by Eliana ], [ ...",True,506
98539,Q2717,"** place). Births. March 25 - Matthew Barney,...",** July 29 **,"** [[ Q17982661 , 29 July 2013 , date ], [ Q17...",True,551


In [6]:
input_text = list(training_data['question'] + ',' + training_data['entity'] + ',' + training_data['wikidata_reply'])
input_text[0]

'** Who was involved in the sheemore ambush? **,**  sheemore ambush  **,** [[ Q7492362 , Sheemore ambush , ambush during the Irish War of Independence ]] **'

In [7]:
target_text = list(training_data['qid'])
target_text[0]

'Q7492362'

In [8]:
X_train_tokenized = tokenizer(['nel: ' + sequence for sequence in input_text], 
                              padding=True, 
                              truncation=True, 
                              max_length=max_source_length)

y_train_tokenized = tokenizer(target_text, 
                              padding=True, 
                              truncation=True, 
                              max_length=max_target_length)

print(len(training_data))
# print(len(training_sample))

130395


In [9]:
val_data = pd.read_csv('./2-NEL_Data/2-csv_format_2/val_data_shuffled.csv')
val_data = val_data.sample(frac=1, random_state=1)
val_data

,qid,question,entity,wikidata_reply,qid_in_reply,input_len
13761,Q497,"** arises from the anus, the distal orifice o...",** anus **,"** [[ Q31785909 , Anus , mountain in Namibia ]...",True,564
12966,Q188,** Hiwi is a German abbreviation. It has two m...,** German **,"** [[ Q188 , German , West Germanic language s...",True,495
135,Q11865151,** what country is ismo kallio from **,** ismo kallio **,"** [[ Q11865151 , Ismo Kallio , Finnish actor ...",True,122
13518,Q165654,** The fire was first spotted at 8:04 p.m. by...,** constable **,"** [[ Q159297 , John Constable , English paint...",True,628
10510,Q81136,** the British Admiralty lost interest in the...,** Northwest Passage **,"** [[ Q17114554 , Northwest Passage , former b...",True,695
...,...,...,...,...,...,...
905,Q7697846,** what type of film is telstar: the joe meek ...,** telstar: the joe meek story **,"** [[ Q7697846 , Telstar: The Joe Meek Story ,...",True,168
5192,Q252,** Bunguran is a small archipelago of Indonesi...,** Indonesia **,"** [[ Q96708780 , Indonesia , scientific journ...",True,543
12172,Q5758978,** American High is a documentary television s...,** Highland Park High School **,"** [[ Q5758983 , Highland Park High School , p...",True,689
235,Q2605094,** What type of music is the album hours? **,** hours **,"** [[ Q25235 , hour , unit of time ], [ Q15704...",True,502


In [10]:
input_text_val = list(val_data['question'] + ',' + val_data['entity'] + ',' + val_data['wikidata_reply'])
input_text_val[0]

'**  arises from the anus, the distal orifice of the gastrointestinal tract. It is a distinct entity from the more common colorectal cancer. The etiology, risk factors, clinical progression, staging,  **,**  anus  **,** [[ Q31785909 , Anus , mountain in Namibia ], [ Q497 , anus , digestive track waste expulsion opening ], [ Q23855 , Anus , Oceanic language spoken in Indonesia ], [ Q20685927 , Anus , album by Alaska Thunderfuck 5000 ], [ Q25016777 , Pulau Anus , island in Papua, Indonesia ], [ Q26235245 , Anus , village in Sarmi Regency, Papua, Indonesia ]] **'

In [11]:
target_text_val = list(val_data['qid'])
target_text_val[0]

'Q497'

In [12]:
X_val_tokenized = tokenizer(['nel: ' + sequence for sequence in input_text_val], 
                              padding=True, 
                              truncation=True, 
                              max_length=max_source_length)

y_val_tokenized = tokenizer(target_text_val, 
                              padding=True, 
                              truncation=True, 
                              max_length=max_target_length)

print(len(val_data))
# print(len(training_sample))

14406


In [13]:
train_dataset = Dataset(X_train_tokenized, y_train_tokenized)

In [14]:
val_dataset = Dataset(X_val_tokenized, y_val_tokenized)

In [15]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    "NEL_model_shuffled_add_spaces_in_input",
    evaluation_strategy ='steps',
    eval_steps = 1000, # Evaluation and Save happens every 50 steps
#     logging_steps = 1000,
    save_steps = 1000,
    save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    learning_rate = 1e-3,
    
    gradient_accumulation_steps =2,
    
    adam_epsilon = 1e-8,
    num_train_epochs = 5,
    report_to="wandb",
#     metric_for_best_model = 'f1',
    load_best_model_at_end=True
)

In [16]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model, 
    args=training_args, 
    train_dataset= train_dataset,
    eval_dataset = val_dataset,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
)

In [17]:
trainer.train()

/home/HadyElkady/miniconda3/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 130395
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 20375
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


/home/HadyElkady/miniconda3/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
1000,0.116600,0.089147
2000,0.082100,0.072329
3000,0.068800,0.065741
4000,0.065200,0.058313
5000,0.049000,0.061230
6000,0.051700,0.057838
7000,0.048000,0.049316


***** Running Evaluation *****
  Num examples = 14406
  Batch size = 16
Saving model checkpoint to NEL_model_shuffled_add_spaces_in_input/checkpoint-1000
Configuration saved in NEL_model_shuffled_add_spaces_in_input/checkpoint-1000/config.json
Model weights saved in NEL_model_shuffled_add_spaces_in_input/checkpoint-1000/pytorch_model.bin
/home/HadyElkady/miniconda3/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 14406
  Batch size = 16
Saving model checkpoint to NEL_model_shuffled_add_spaces_in_input/checkpoint-2000
Configuration saved in NEL_model_shuffled_add_spaces_in_input/checkpoint-2000/config.json
Model weights saved in NEL_model_shuffled_add_spaces_in_input/checkpoint-2000/pytorch_model.bin
/home/HadyElkady/miniconda3/li

In [18]:
print('finished')

finished


In [19]:
# Num examples = 130395
# Num Epochs = 5
# Instantaneous batch size per device = 4
# Total train batch size (w. parallel, distributed & accumulation) = 12
# Gradient Accumulation steps = 1
# Total optimization steps = 54335